In [13]:
import numpy as np
import scipy as sp
from scipy import linalg


- assume double integrator model

In [2]:
# M players
M = 3

# N time steps
N = 10

# n state size
n = 4 # (x, y, v_x, v_y)

# m control input size
m = 2 # (a_x, a_y)

# timestep
dt = 0.01

### States

$X^v$ and $U^v$ trajectory for each player $v$ basically like the CFTOCP batch approach without substitution from HW1

$X^v = [x_1, ... x_N]$ and $U^v = [u_0, ... u_{N-1}]$

For the full Nash equilibium problem, just concatenate all the players trajectories together 

$X = [X^1, X^2, ..., X^m]$ and $U = [U^1, U^2, ..., U^m]$



### Dynamics

$D(X,U)$ contains all trajectories and control inputs stacked

In [ ]:
# double integrator
A = np.eye(n) + np.diag([dt, dt], k=int(n / 2))
B = np.vstack((np.eye(m) * dt**2 / 2, (np.eye(m) * dt)))

# player dynamics
# A_eq * X_v + B_eq * U_v = E_eq
a_block = linalg.block_diag(*([-A] * (N - 1)))
A_eq = np.eye(N * n) + np.pad(a_block, [(n, 0), (0, n)])
B_eq = linalg.block_diag(*([-B] * N))
E_eq = np.vstack((A, np.zeros(((N - 1) * n, n))))

# system dynamics for all players
# A_sys * X + B_sys * U = E_sys
A_sys = linalg.block_diag(*([A_eq] * M))
B_sys = linalg.block_diag(*([B_eq] * M))
E_sys = np.vstack(([E_eq] * M))

(120, 120)